In [1]:
# LOAD PACKAGES
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import haversine as hs
import requests
import json
import urllib

In [2]:
# User address info (example)
STREET = "777 6TH ST.NW, SUITE 520"
CITY = "WASHINGTON"
STATE = "DC"
POSTALCODE = "20001"

# Read in facility data and filter to user state
ft_df = pd.read_csv("C:/Users/catherine.sun/OneDrive - Summit, LLC/Documents/Projects/DataJam 2023/Datasets/FindTreatment_Facility_listing_DMV_2023_04_03.csv")

In [127]:
# Define distance_from function which outputs the distance between two coordinates (great circle distance in miles)
def distance_from(loc1,loc2): 
    dist=hs.haversine(loc1, loc2, unit=hs.Unit.MILES)
    return round(dist,2)

In [131]:
def find_coord(STREET, CITY, STATE, POSTALCODE, RANGE, facility_data):

    # Creat dictionary and full address string of user address info
    user_dict = {'street' : STREET, 'city' : CITY, 'state' : STATE, 'postalcode' : POSTALCODE}
    loc = user_dict['street'] +str(",")+ user_dict['city'] +str(",")+ user_dict['state'] +str(",")+ user_dict['postalcode']
    loc = loc.upper()

    # Filter facility data to user state
    ft_df_filt = facility_data.query('state == @STATE')

    # Set Nominatim username
    locator = Nominatim(user_agent="summit")

    # Geocoding user address
    try:
        struc_query = locator.geocode(user_dict)
        user_dict['latitude'] = struc_query.latitude
        user_dict['longitude'] = struc_query.longitude
        user_dict['method'] = 'structured'
    except:
        try:
            simple_query = locator.geocode(loc)
            user_dict['latitude'] = simple_query.latitude
            user_dict['longitude'] = simple_query.longitude
            user_dict['method'] = 'simple'
        except Exception as err:
            return(print("Cannot locate address: " + str(err)))

    # Concatenating lat and long to create a consolidated location as accepted by havesine function
    user_dict['coor'] = tuple((user_dict['latitude'], user_dict['longitude']))
    ft_df_filt['coor'] = list(zip(ft_df_filt.latitude, ft_df_filt.longitude))

    # Find distance from user to each facility and filter for facilities within 5 miles (user edits dist range)
    ft_df_filt['distance']=ft_df_filt['coor'].apply(lambda x: distance_from(user_dict['coor'],x))
    ft_df_sort = ft_df_filt.sort_values('distance').reset_index(drop=True).query('distance <= @RANGE')

    # Return data of closest facilities within given range
    return(ft_df_sort)

In [132]:
find_coord(STREET, CITY, STATE, POSTALCODE, 5, ft_df)

Cannot locate address: Non-successful status code 502


Code blocks of each section from function

In [ ]:
user_dict = {'street' : STREET, 'city' : CITY, 'state' : STATE, 'postalcode' : POSTALCODE}
loc = user_dict['street'] +str(",")+ user_dict['city'] +str(",")+ user_dict['state'] +str(",")+ user_dict['postalcode']
loc = loc.upper()

In [ ]:
# Read in facility data and filter to user state
ft_df = pd.read_csv("C:/Users/catherine.sun/OneDrive - Summit, LLC/Documents/Projects/DataJam 2023/Datasets/FindTreatment_Facility_listing_DMV_2023_04_03.csv")
ft_df_filt = ft_df.query('state == @STATE')

In [130]:
# Set Nominatim username
locator = Nominatim(user_agent="summit")

# Geocoding user address
try:
    struc_query = locator.geocode(user_dict)
    user_dict['latitude'] = struc_query.latitude
    user_dict['longitude'] = struc_query.longitude
    user_dict['method'] = 'structured'
except:
    try:
        simple_query = locator.geocode(loc)
        user_dict['latitude'] = simple_query.latitude
        user_dict['longitude'] = simple_query.longitude
        user_dict['method'] = 'simple'
    except Exception as err:
        print("Cannot locate address: " + str(err))

Cannot locate address: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=777+6TH+ST.NW%2C+SUITE+520%2CWASHINGTON%2CDC%2C20001&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


In [ ]:
# Concatenating lat and long to create a consolidated location as accepted by havesine function
user_dict['coor'] = tuple((user_dict['latitude'], user_dict['longitude']))
ft_df_filt['coor'] = list(zip(ft_df_filt.latitude, ft_df_filt.longitude))

In [ ]:
# Find distance from user to each facility and filter for facilities within 5 miles (user edits dist range)
ft_df_filt['distance']=ft_df_filt['coor'].apply(lambda x: distance_from(user_dict['coor'],x))
ft_df_sort = ft_df_filt.sort_values('distance').reset_index(drop=True).query('distance <= 5')

In [ ]:
ft_df_sort[['name','street1','street2','city','state','zip','zip4','county',
            'phone','intake_prompt','intake1','intake2','website',
            'latitude','longitude','distance','score']]

In [21]:
url = "https://geocoding.geo.census.gov/geocoder/locations/address?street={0}&city={1}&state={2}&zip={3}&benchmark=Public_AR_Current&format=json".format(STREET, CITY, STATE, POSTALCODE)
url.replace(" ","%20")

'https://geocoding.geo.census.gov/geocoder/locations/address?street=777%206TH%20ST.NW,%20SUITE%20520&city=WASHINGTON&state=DC&zip=20001&benchmark=Public_AR_Current&format=json'

In [22]:
r = requests.get(url)
rr = json.loads(r.content)

In [35]:
rr['result']['addressMatches'][0]['coordinates']

{'x': -77.01989552676964, 'y': 38.89940834160424}